# <center> MobileNetV2 x1.4 Adversarial Training on Generated Examples </center>

### Library Definitions.

We use seed 0 throughout our codes to ensure reproducibility.

In [ ]:
from torch.quantization import MovingAverageMinMaxObserver
from torch.ao.quantization.observer import MinMaxObserver
from torch.quantization import QuantStub, DeQuantStub
import torch.nn as nn
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.transforms.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from torchvision import datasets
from art.estimators.classification import PyTorchClassifier
from art.utils import load_cifar10
from art.preprocessing.standardisation_mean_std import StandardisationMeanStdPyTorch
from art.attacks.evasion.hop_skip_jump import HopSkipJump
import time
import pickle
import os
from math import log10, sqrt
import cv2
import random
from torch.utils.data import DataLoader
torch.manual_seed(0)
torch.cuda.manual_seed(0)
np.random.seed(0)
random.seed(0)


### Metric Definitions.

Standard Average meter and test function along with a custom testing function "accuracy".
Training function is included later on.

In [ ]:

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)





def test(model: nn.Module, dataloader: DataLoader, cuda=False) -> float:
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for data in dataloader:
            inputs, labels = data
            if cuda:
              inputs = inputs.cuda()
              labels = labels.cuda()
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

def accuracy(output, target):
    """ Computes the top 1 accuracy """
    with torch.no_grad():
        batch_size = target.size(0)

        _, pred = output.topk(1, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        correct_one = correct[:1].view(-1).float().sum(0, keepdim=True)
        return correct_one.mul_(100.0 / batch_size).item()

### Project specific functions.

"normalize_np" Applies the same Mean and Std of the tensor CIFAR-10 Dataset to the Numpy one.

In [ ]:
def PSNR(original, compressed):
    mse = np.mean((original - compressed) ** 2)
    epsilon = 1e-10  # Small epsilon to avoid division by zero
    mse = max(mse, epsilon)  # Ensure MSE is not zero
    max_pixel = 1.0
    psnr = 20 * log10(max_pixel / sqrt(mse))
    return psnr

def show_image(img):
    plt.imshow(img)
    plt.show(block=False)
    
def normalize_np(img):
  img = torch.from_numpy(img)
  img = F.normalize(img, [0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])
  return img.numpy()

### Dataset Preparation

In [ ]:
#Load the CIFAR10 dataset from ART

(x_train, y_train), (x_test, y_test), min_pixel_value, max_pixel_value = load_cifar10()

#Convert to PyTorch's NCHW format

x_train = np.transpose(x_train, (0, 3, 1, 2)).astype(np.float32)
x_test = np.transpose(x_test, (0, 3, 1, 2)).astype(np.float32)


# also Load the Pytorch CIFAR-10 tensor dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,shuffle=True, num_workers=16, pin_memory=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64,shuffle=False, num_workers=16, pin_memory=True)

# Set image labels
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


# Sanity Check: Show first 5 images of CIFAR-10 (numpy version)
printable = torchvision.utils.make_grid(torch.from_numpy(x_train[0:5])).numpy()
printable = np.transpose(printable, (1, 2, 0) )
show_image(printable)

# Sanity Check: Include the adjusted versions (should be "high contrast")
printable_norm = torchvision.utils.make_grid(torch.from_numpy(normalize_np(x_train)[0:5])).numpy()
printable_norm = np.transpose(printable_norm, (1, 2, 0) )
show_image(printable_norm)


### Adversarial Examples Dataset

Here you have 2 options.
1. Download the datasets manually and update the paths.
2. Let the notebook download and load them.

Run the corresponding block.

1. Download Manually

In [ ]:
import os

# Adversarial Set 1 https://drive.google.com/file/d/1NdiiJ2hBOuzU7YZyK1p0dNkWlbHriqLD/view?usp=sharing

# Adversarial Set 2 https://drive.google.com/file/d/14GE6vJhgUJvy5SLerDdy1V0OEVWDOryd/view?usp=sharing

# Adversarial Set 3 https://drive.google.com/file/d/1NGyjybPJpXaRPsKfICI0vN3CbADB--o5/view?usp=sharing


#Your path to downloaded files
path = "~/Downloads/"

file1 = open(os.path.join(path, "train_mobilenet_hop_cifar_for_1it.pkl"),'rb')
file2 = open(os.path.join(path, "train_mobilenet_hop_cifar_for_2it.pkl"),'rb')
file3 = open(os.path.join(path, "train_mobilenet_hop_cifar_for_3it.pkl"),'rb')

adversarial_1 = pickle.load(file1)
adversarial_2 = pickle.load(file2)
adversarial_3 = pickle.load(file3)

2. Load by Downloading

In [ ]:
import urllib

adversarial_1 = pickle.load(urllib.request.urlopen("https://drive.usercontent.google.com/download?id=1NdiiJ2hBOuzU7YZyK1p0dNkWlbHriqLD&export=download&confirm=t&uuid=0"))
adversarial_2 = pickle.load(urllib.request.urlopen("https://drive.usercontent.google.com/download?id=14GE6vJhgUJvy5SLerDdy1V0OEVWDOryd&export=download&confirm=t&uuid=0"))
adversarial_3 = pickle.load(urllib.request.urlopen("https://drive.usercontent.google.com/download?id=1NGyjybPJpXaRPsKfICI0vN3CbADB--o5&export=download&confirm=t&uuid=0"))

#### Dataset Definitions

In [ ]:
def custom_collate(batch):
    # Combine a list of samples into a batch
    data, labels = zip(*batch)
    data = torch.stack(data)
    labels = torch.tensor(labels, dtype=torch.long)
    return data, labels

First Adversarial Example dataset (30k examples)

In [ ]:
sample_size = 30000

x_train_set = adversarial_1[0:sample_size]
y_train_set = np.empty((sample_size)).astype(np.int8)

# Remove 1 hot encoding
for i in range(sample_size):
    y_train_set[i] = np.where(y_train[i] == 1)[0][0]
    
print(x_train_set.shape, y_train_set.shape)


trainingSet = torch.utils.data.TensorDataset(torch.from_numpy(normalize_np(x_train_set)).type(torch.FloatTensor), torch.from_numpy(y_train_set).type(torch.LongTensor))

train_dev_sets = torch.utils.data.ConcatDataset([trainingSet])



retrain_loader_1 = torch.utils.data.DataLoader(dataset=train_dev_sets, batch_size=64, num_workers=16, pin_memory=True, shuffle=True, collate_fn=custom_collate)

Second Adversarial Example dataset (20k examples)

In [ ]:
sample_size = 20000

x_train_set = adversarial_2[0:sample_size]
y_train_set = np.empty((sample_size)).astype(np.int8)

# Remove 1 hot encoding
for i in range(sample_size):
    y_train_set[i] = np.where(y_train[i] == 1)[0][0]
    
print(x_train_set.shape, y_train_set.shape)


trainingSet = torch.utils.data.TensorDataset(torch.from_numpy(normalize_np(x_train_set)).type(torch.FloatTensor), torch.from_numpy(y_train_set).type(torch.LongTensor))

train_dev_sets = torch.utils.data.ConcatDataset([trainingSet])



retrain_loader_2 = torch.utils.data.DataLoader(dataset=train_dev_sets, batch_size=64, num_workers=16, pin_memory=True, shuffle=True, collate_fn=custom_collate)

Third Adversarial Example dataset (10k examples)

In [ ]:
sample_size = 10000

x_train_set = adversarial_3[0:sample_size]
y_train_set = np.empty((sample_size)).astype(np.int8)

# Remove 1 hot encoding
for i in range(sample_size):
    y_train_set[i] = np.where(y_train[i] == 1)[0][0]
    
print(x_train_set.shape, y_train_set.shape)


trainingSet = torch.utils.data.TensorDataset(torch.from_numpy(normalize_np(x_train_set)).type(torch.FloatTensor), torch.from_numpy(y_train_set).type(torch.LongTensor))

train_dev_sets = torch.utils.data.ConcatDataset([trainingSet])



retrain_loader_3 = torch.utils.data.DataLoader(dataset=train_dev_sets, batch_size=64, num_workers=16, pin_memory=True, shuffle=True, collate_fn=custom_collate)

### Model Retraining

Training process is split into three steps.
Here you also have 2 choices.

In this notebook, we start from the initial model and retrain it 3 times.

On "Adver_Train_separate" folder, we include the retraining process split in 3 steps where we load the already trained version in each one.
Ideally we want them separately to retrieve the trained model and generate adversarial examples on it for our next iteration.

In [ ]:
#----------------------------------------------------------------------     FIRST ADVERSARIAL TRAINING ITERATION ----------------------------------------------------------------------

def train(model: nn.Module, advloader: DataLoader,testloader: DataLoader, cuda=False):

    #The flag's purpose is to stop the model on the previously achieved accuracies. Feel free to change/remove it.
    flagged = False

    #For the first iteration we use SGD optimizer. Later on we switch to Adam
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9, dampening=0,weight_decay=0.0005, nesterov=True)
    model.train()

    for epoch in range(400):  # Define your max training epochs
        if flagged:
            break   # Training pause flag on pre-determined accuracies.
        else:
            #Typical training procedure
            running_loss = AverageMeter('loss')
            acc = AverageMeter('train_acc')
            for i, data in enumerate(advloader, 0):
        
                inputs, labels = data
                if cuda:
                  inputs = inputs.cuda()
                  labels = labels.cuda()
                # zero the parameter gradients
                optimizer.zero_grad()
                # forward + backward + optimize
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                # print statistics
                running_loss.update(loss.item(), outputs.shape[0])
                acc.update(accuracy(outputs, labels), outputs.shape[0])
                if i % 100 == 0:    # print every 100 mini-batches
                    print('[%d, %5d] ' %
                        (epoch + 1, i + 1), running_loss, acc)
                    
            #--------- REMOVABLE PART ----------
            #Test model after each epoch on test and adversarial sets

            score = test(model, testloader, cuda=True)
            score1 = test(model, advloader, cuda=True)
            print('Current Epoch {} : Test Set accuracy: {}% - FP32'.format(epoch, score))
            print('Current Epoch {} : Adversarial Example set accuracy: {}% - FP32'.format(epoch, score1))
            #  IF x and y scores achieved, stop training.
            if (score >= 94.10 and score1 >= 99.99):
                flagged = True
            model.train()

            #   NOTE: THIS CAN AND WILL SUBSTANTIALLY AFFECT TRAINING TIME AND PERFORMANCE.
            #   If you want the achieved accuracies its better to load the intermediate model weights provided.

            #-----------------------------------

    print('Finished Training')



#1st retraining: Load pretrained model.

model = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar10_mobilenetv2_x1_4", pretrained=True)
model.to("cuda")
model.eval()

#Initial Accuracy
score = test(model, testloader, cuda=True)
print('Initial accuracy of the network on test images: {}% - FP32'.format(score))

train(model,retrain_loader_1,testloader,cuda=True)

#After training
score = test(model, testloader, cuda=True)
print('After 1 Adversarial Training of the network: Accuracy on test images: {}% - FP32'.format(score))

#Save model for later use
torch.save(model.state_dict(), "MobileNetV2_x1_4_AdvTrain1_CIF10_{}acc.pkl".format(score))
#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------



#----------------------------------------------------------------------     SECOND ADVERSARIAL TRAINING ITERATION ----------------------------------------------------------------------

def train(model: nn.Module, advloader: DataLoader,testloader: DataLoader, cuda=False):

    #The flag's purpose is to stop the model on the previously achieved accuracies. Feel free to change/remove it.
    flagged = False

    #Switch to Adam increasing generalization
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    model.train()

    for epoch in range(400):  # Define your max training epochs
        if flagged:
            break   # Training pause flag on pre-determined accuracies.
        else:
            #Typical training procedure
            running_loss = AverageMeter('loss')
            acc = AverageMeter('train_acc')
            for i, data in enumerate(advloader, 0):
        
                inputs, labels = data
                if cuda:
                  inputs = inputs.cuda()
                  labels = labels.cuda()
                # zero the parameter gradients
                optimizer.zero_grad()
                # forward + backward + optimize
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                # print statistics
                running_loss.update(loss.item(), outputs.shape[0])
                acc.update(accuracy(outputs, labels), outputs.shape[0])
                if i % 100 == 0:    # print every 100 mini-batches
                    print('[%d, %5d] ' %
                        (epoch + 1, i + 1), running_loss, acc)
                    
            #--------- REMOVABLE PART ----------
            #Test model after each epoch on test and adversarial sets

            score = test(model, testloader, cuda=True)
            score1 = test(model, advloader, cuda=True)
            print('Current Epoch {} : Test Set accuracy: {}% - FP32'.format(epoch, score))
            print('Current Epoch {} : Adversarial Example set accuracy: {}% - FP32'.format(epoch, score1))
            #  IF x and y scores achieved, stop training.
            if (score >= 93.55 and score1 >= 99.98):
                flagged = True
            model.train()

            #   NOTE: THIS CAN AND WILL SUBSTANTIALLY AFFECT TRAINING TIME AND PERFORMANCE.
            #   If you want the achieved accuracies its better to load the intermediate model weights provided.

            #-----------------------------------

    print('Finished Training')



#2nd retraining: Model Loaded.
model.eval()

#Initial Accuracy
score = test(model, testloader, cuda=True)
print('Sanity Check: Adversarial Training 1: accuracy on test images: {}% - FP32'.format(score))

train(model,retrain_loader_2,testloader,cuda=True)

#After training
score = test(model, testloader, cuda=True)
print('After 2nd Adversarial Training of the network: Accuracy on test images: {}% - FP32'.format(score))

#Save model for later use
torch.save(model.state_dict(), "MobileNetV2_x1_4_AdvTrain2_CIF10_{}acc.pkl".format(score))
#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------



#----------------------------------------------------------------------     THIRD ADVERSARIAL TRAINING ITERATION ----------------------------------------------------------------------

def train(model: nn.Module, advloader: DataLoader,testloader: DataLoader, cuda=False):

    #The flag's purpose is to stop the model on the previously achieved accuracies. Feel free to change/remove it.
    flagged = False

    #Keep Adam optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=5e-7)
    model.train()

    for epoch in range(400):  # Define your max training epochs
        if flagged:
            break   # Training pause flag on pre-determined accuracies.
        else:
            #Typical training procedure
            running_loss = AverageMeter('loss')
            acc = AverageMeter('train_acc')
            for i, data in enumerate(advloader, 0):
        
                inputs, labels = data
                if cuda:
                  inputs = inputs.cuda()
                  labels = labels.cuda()
                # zero the parameter gradients
                optimizer.zero_grad()
                # forward + backward + optimize
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                # print statistics
                running_loss.update(loss.item(), outputs.shape[0])
                acc.update(accuracy(outputs, labels), outputs.shape[0])
                if i % 100 == 0:    # print every 100 mini-batches
                    print('[%d, %5d] ' %
                        (epoch + 1, i + 1), running_loss, acc)
                    
            #--------- REMOVABLE PART ----------
            #Test model after each epoch on test and adversarial sets

            score = test(model, testloader, cuda=True)
            score1 = test(model, advloader, cuda=True)
            print('Current Epoch {} : Test Set accuracy: {}% - FP32'.format(epoch, score))
            print('Current Epoch {} : Adversarial Example set accuracy: {}% - FP32'.format(epoch, score1))
            #  IF x and y scores achieved, stop training.
            if (score >= 92.75 and score1 >= 98.70):
                flagged = True
            model.train()

            #   NOTE: THIS CAN AND WILL SUBSTANTIALLY AFFECT TRAINING TIME AND PERFORMANCE.
            #   If you want the achieved accuracies its better to load the intermediate model weights provided.

            #-----------------------------------

    print('Finished Training')



#3rd retraining: Inplace model.
model.eval()

#Initial Accuracy
score = test(model, testloader, cuda=True)
print('Sanity Check: Adversarial Training 2: accuracy on test images: {}% - FP32'.format(score))

train(model,retrain_loader_3,testloader,cuda=True)

#After training
score = test(model, testloader, cuda=True)
print('After 3rd Adversarial Training of the network: Accuracy on test images: {}% - FP32'.format(score))
#Save model for later use
torch.save(model.state_dict(), "MobileNetV2_x1_4_AdvTrain3_CIF10_{}acc.pkl".format(score))
#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------